In [1]:
import sys

sys.path.append("../")
import os.path as osp
from stanza_utils import *
from glob import glob
from tqdm import tqdm

pos, dep, tag = {}, {}, {}
for file in glob("../data/**.train.txt"):
    domain = osp.basename(file).split('.')[0]
    pos_list, dep_list, tag_list = [], [], []
    lines = open(file, "r").read().splitlines()
    sentences_list = annotation_plus([line.split("***")[0] for line in lines])
    label_list = [line.split("***")[1] for line in lines]
    for sentence, labels in tqdm(zip(sentences_list, label_list), total=len(lines), desc=domain):
        for token, label in zip(sentence.to_dict(), labels.split(' ')):
            pos_list.append(token['xpos'])
            dep_list.append(token['deprel'])
            tag_list.append(label)
    pos[domain] = pos_list
    dep[domain] = dep_list
    tag[domain] = tag_list

/root/miniconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-19 18:45:40 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2023-05-19 18:45:40 INFO: Using device: cuda
2023-05-19 18:45:40 INFO: Loading: tokenize
2023-05-19 18:45:40 INFO: Loading: pos
2023-05-19 18:45:44 INFO: Loading: lemma
2023-05-19 18:45:44 INFO: Loading: depparse
2023-05-19 18:45:44 INFO: Done loading processors!
service: 100%|██████████| 1492/1492 [00:00<00:00, 8520.99it/s]


In [12]:
import pandas as pd
from pandas import ExcelWriter
 
domains = pos.keys()
total = 0


def func(x: pd.Series):
    if x.name == "tag": return len(x[x != 'O']) * 1.0 / len(x)
    else: return len(x[x != 'O']) * 1.0 / total


writer = ExcelWriter("./annotation.xlsx")
pos_writer = ExcelWriter("./pos.xlsx")
dep_writer = ExcelWriter("./dep.xlsx")
for i, domain in enumerate(domains):
    data = pd.DataFrame({"pos": pos[domain], "dep": dep[domain], "tag": tag[domain]})
    total = len(data[data['tag'] != 'O'])
    data['tag_count'] = data['tag'].copy()
    groups = data.groupby(["pos", "dep"], group_keys=False)
    df = groups.agg(func).sort_values(["tag_count", "tag"], ascending=False)
    df.to_excel(writer, sheet_name=domain)
    pos_df = df.groupby("pos").agg({
        "tag_count": "sum"
    }).sort_values("tag_count", ascending=False).head(15)
    pos_df.to_excel(pos_writer, sheet_name=domain)
    dep_df = df.groupby("dep").agg({
        "tag_count": "sum"
    }).sort_values("tag_count", ascending=False).head(15)
    dep_df.to_excel(dep_writer, sheet_name=domain)
writer.close()
pos_writer.close()
dep_writer.close()